In [106]:
import lxml.etree as etree
import lxml.html as HM
import re
from io import BytesIO

In [107]:
def replace_xmlns(xml_string):
    """
    chagnes xmlns to tag:item to tag:tag
    >>> test_xmlns = 'xmlns:excerpt="http://wordpress.org/export/1.2/excerpt/"'
    >>> replace_xmlns(test_xmlns)
    'xmlns:excerpt="excerpt"'
    """
    pre_chan, chan, post_chan= xml_string.partition('<channel>')
    # make this quick, only replace xmlns in prechannel, where it should be
    pre_chan = re.sub(r'xmlns:(?P<label>\w*)\=\"(?P<val>.*)\"',
                         r'xmlns:\g<label>="\g<label>"',
                         pre_chan)
    return pre_chan + chan + post_chan


test_xmlns = '\txmlns:excerpt="http://wordpress.org/export/1.2/excerpt/"'
replace_xmlns(test_xmlns)

'\txmlns:excerpt="excerpt"'

In [108]:
def remove_encoding(xml_string):
    return re.sub(r'.*encoding=.*', '', xml_string)
print(remove_encoding(xml_string).split("<channel>")[0])


<!-- This is a WordPress eXtended RSS file generated by WordPress as an export of your site. -->
<!-- It contains information about your site's posts, pages, comments, categories, and other content. -->
<!-- You may use this file to transfer that content from one site to another. -->
<!-- This file is not intended to serve as a complete backup of your site. -->

<!-- To import this information into a WordPress site follow these steps: -->
<!-- 1. Log in to that site as an administrator. -->
<!-- 2. Go to Tools: Import in the WordPress admin panel. -->
<!-- 3. Install the "WordPress" importer from the list. -->
<!-- 4. Activate & Run Importer. -->
<!-- 5. Upload this file using the form provided on that page. -->
<!-- 6. You will first be asked to map the authors in this export file to users -->
<!--    on the site. For each author, you may choose to map to an -->
<!--    existing user on the site or to create a new user. -->
<!-- 7. WordPress will then import each of the posts, pages,

In [109]:
xml_string = (open('greenkeyintranet.xml').read())
xml_string = replace_xmlns(remove_encoding(xml_string))
root = etree.XML(xml_string)

In [110]:
chan = root[0]
len(chan)

572

In [111]:
items = [e for e in chan.getchildren() if e.tag=='item']

In [112]:
cat_items = [i for i in items if "category" in [e.tag for e in i]]
test_cat_item = cat_items[0].findall("category")[0]
test_cat_item.attrib

{'domain': 'media-category', 'nicename': 'news-press'}

In [113]:
def item_dict(item):
    #{e.tag:e.text for e in item}
    ret_dict = {"terms":[]}
    for e in item:
        if e.tag == "category":
            ret_dict["terms"].append(e.attrib)
            continue
        ret_dict[e.tag] = e.text
    return ret_dict

In [114]:
item_dict_gen = lambda : (item_dict(item) for item in items)

In [115]:
# does every item have the same amount of keys?
keys_list = [set(i.keys()) for i in item_dict_gen()]
all_keys = keys_list[0].union(*keys_list[1:])
shared_keys = keys_list[0].intersection(*keys_list[1:])
optional_keys = all_keys - shared_keys
optional_keys

{'{wp}attachment_url', '{wp}postmeta'}

In [117]:
[e['date'] for e in item_dict_gen()]

KeyError: 'date'

In [96]:
items_with_opt_keys = sorted([(d.get('link'),list(d.keys()-shared_keys)) for d in item_dict_gen() if optional_keys.intersection(d.keys())])

In [97]:
# Can I find all the expected json vals from json import
json_vals = ["slug", # {wp}post_naem
            "ID", # guid?
            "title", #
            "description", #
            "content", #
            "author", # creator?
            "terms", ## WTF??
            "date", # post date?
            "featured_image",]

In [98]:
shared_keys

{'description',
 'guid',
 'link',
 'pubDate',
 'terms',
 'title',
 '{content}encoded',
 '{dc}creator',
 '{excerpt}encoded',
 '{wp}comment_status',
 '{wp}is_sticky',
 '{wp}menu_order',
 '{wp}ping_status',
 '{wp}post_date',
 '{wp}post_date_gmt',
 '{wp}post_id',
 '{wp}post_name',
 '{wp}post_parent',
 '{wp}post_password',
 '{wp}post_type',
 '{wp}status'}

In [113]:
#slug
test_item= next(item_dict_gen())

In [120]:
items_with_img = [item for item in item_dict_gen() if "<img" in item["{content}encoded"]]

In [130]:
## Don't need this probably
def grab_img_urls_from_string(content):
    # try XML?
    if not "<img" in content:
        return None
    try:
        tree = HM.fromstring(content)
    except XMLSyntax:
        print("couldn't parse")
    if tree is not None:
        img_tags = tree.findall(".//img")
        if img_tags is not None:
            return [e.attrib['src'] for e in img_tags] 

assert grab_img_url_item(items_with_img[0]['{content}encoded'])
img_links = [grab_img_urls_from_string(item['{content}encoded']) for item in item_dict_gen() if grab_img_urls_from_string(item['{content}encoded'])]

In [118]:
import json
def translate_item(item_dict):
    """cleanup item keys to match dest format"""
    ret_dict = {}
    # assume we have the values we need, fail hard otherwise
    ret_dict['slug']= item_dict['{wp}post_name']
    ret_dict['ID']= item_dict['guid']
    ret_dict['title']= item_dict['title']
    ret_dict['description']= item_dict['description']
    ret_dict['content']= item_dict['{content}encoded']
    ret_dict['author']= item_dict['{dc}creator']
    # ret_dict['terms']= ""
    ret_dict['date']= item_dict['pubDate']
    # ret_dict['featured_image'] = None
    return ret_dict
translate_item(test_item)

NameError: name 'test_item' is not defined

In [119]:
all_the_data = [translate_item(item) for item in item_dict_gen()]

In [121]:
date_format = all_the_data[0]['date']

In [122]:
date_format

'Tue, 19 Nov 2013 22:26:49 +0000'

In [127]:
import datetime
import time
import calendar

In [153]:
def convert_date(d, custom_date_string=None):
    try:
        date =  time.strftime("%m-%d-%Y", time.strptime(d[:16], '%a, %d %b %Y'))
    except ValueError:
        date = custom_date_string or datetime.datetime.today().strftime("%m-%d-%Y")
    return date

In [161]:
[e for e in all_the_data if not e.get('title')]

[{'ID': 'http://intranet.greenkeyllc.com/?p=120',
  'author': 'randib@greenkeyllc.com',
  'content': ' ',
  'date': 'Fri, 30 Nov 2012 17:46:14 +0000',
  'description': None,
  'slug': '120',
  'title': None},
 {'ID': 'http://intranet.greenkeyllc.com/?p=121',
  'author': 'randib@greenkeyllc.com',
  'content': ' ',
  'date': 'Fri, 30 Nov 2012 17:46:14 +0000',
  'description': None,
  'slug': '121',
  'title': None},
 {'ID': 'http://intranet.greenkeyllc.com/?p=122',
  'author': 'randib@greenkeyllc.com',
  'content': ' ',
  'date': 'Fri, 30 Nov 2012 17:46:14 +0000',
  'description': None,
  'slug': '122',
  'title': None},
 {'ID': 'http://intranet.greenkeyllc.com/?p=123',
  'author': 'randib@greenkeyllc.com',
  'content': ' ',
  'date': 'Fri, 30 Nov 2012 17:46:14 +0000',
  'description': None,
  'slug': '123',
  'title': None}]

In [168]:
categories = [e for e in chan.getchildren() if 'category' in e.tag]
cat = categories[0]

In [195]:
def get_category_dict(chan):
    cats = [e for e in chan.getchildren() if '{wp}category' in e.tag]
    cats_dict = {}
    for e in cats:
        slug = e.find('.//{wp}category_nicename').text
        cats_dict['slug'] = {'slug':cat.find("./{wp}category_nicename").text, 
                             'name':cat.find("./{wp}cat_name").text}
        cats_dict['taxonomy'] = 'category'
        return cats_dict
        
get_category_dict(chan)

{'slug': {'name': 'Employee News Articles', 'slug': 'employee-news-articles'},
 'taxonomy': 'category'}

In [185]:
def get_terms_dict(chan):
    terms = [e for e in chan.getchildren() if 'term' in e.tag]
    terms_dict = {}
    for e in terms:
        slug = e.find('.//{wp}term_slug').text
        terms_dict[slug] = {'slug':slug}
        name = e.find('.//{wp}term_name').text # need some regex parsing here
        terms_dict[slug]['name'] = name
        terms_dict[slug]['taxonomy'] = 'post-tag'
    return terms_dict
get_terms_dict(chan)

{'401k': {'name': '401K', 'slug': '401k', 'taxonomy': 'media-category'},
 '401k-enrollment-forms': {'name': '401K Enrollment Forms',
  'slug': '401k-enrollment-forms',
  'taxonomy': 'media-category'},
 '401k-faq': {'name': '401K Faq',
  'slug': '401k-faq',
  'taxonomy': 'media-category'},
 '401k-summary-plan-documents': {'name': '401K Summary Plan Documents',
  'slug': '401k-summary-plan-documents',
  'taxonomy': 'media-category'},
 'benefits-and-policies': {'name': 'Benefits and Policies',
  'slug': 'benefits-and-policies',
  'taxonomy': 'media-category'},
 'cheat-sheets': {'name': 'Cheat Sheets',
  'slug': 'cheat-sheets',
  'taxonomy': 'media-category'},
 'cigna-dental-election-forms': {'name': 'Cigna Dental Election Forms',
  'slug': 'cigna-dental-election-forms',
  'taxonomy': 'media-category'},
 'cigna-dental-enrollment-forms': {'name': 'Cigna Dental Enrollment Forms',
  'slug': 'cigna-dental-enrollment-forms',
  'taxonomy': 'media-category'},
 'cigna-dental-faq': {'name': 'Cigna 